In [2]:
!pip install python-pkcs11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-pkcs11]


In [2]:
!softhsm2-util --show-slots

Available slots:
Slot 936525387
    Slot info:
        Description:      SoftHSM slot ID 0x37d23e4b                                      
        Manufacturer ID:  SoftHSM project                 
        Hardware version: 2.6
        Firmware version: 2.6
        Token present:    yes
    Token info:
        Manufacturer ID:  SoftHSM project                 
        Model:            SoftHSM v2      
        Hardware version: 2.6
        Firmware version: 2.6
        Serial number:    e28305cdb7d23e4b
        Initialized:      yes
        User PIN init.:   yes
        Label:            DemoToken                       
Slot 1
    Slot info:
        Description:      SoftHSM slot ID 0x1                                             
        Manufacturer ID:  SoftHSM project                 
        Hardware version: 2.6
        Firmware version: 2.6
        Token present:    yes
    Token info:
        Manufacturer ID:  SoftHSM project                 
        Model:            SoftHSM v2

In [1]:
# brew install softhsm opensc
#
# mkdir -p ~/softhsm/tokens
# echo "directories.tokendir = $HOME/softhsm/tokens" > ~/softhsm/softhsm2.conf
# export SOFTHSM2_CONF=$HOME/softhsm/softhsm2.conf
#
# softhsm2-util --init-token --slot 0 --label "DemoToken" --so-pin 1234 --pin 5678

import os
import pkcs11
from pkcs11 import KeyType, Mechanism

os.environ['SOFTHSM2_CONF'] = os.path.expanduser('~/softhsm/softhsm2.conf')

# Path to the SoftHSM shared library
# lib_path = "/usr/local/lib/softhsm/libsofthsm2.so" 
lib_path = "/opt/homebrew/lib/softhsm/libsofthsm2.so"

try:
    lib = pkcs11.lib(lib_path)
    
    # Let's list all tokens to be sure what we have
    tokens = lib.get_tokens()
    print(f"Available tokens: {[t.label for t in tokens]}")
    
    # Now try to get the specific one
    token = lib.get_token(token_label='DemoToken')
    print(f"Connected to: {token.label}")
    
except pkcs11.NoSuchToken:
    print("Error: Could not find 'DemoToken'. Check labels listed above.")
except Exception as e:
    print(f"Unexpected error: {e}")
    
# 1. Load the PKCS#11 Library
lib = pkcs11.lib(lib_path)
token = lib.get_token(token_label='DemoToken')

print(f"Connecting to token: {token.label}")

# 2. Open a session and login with your PIN
with token.open(user_pin='5678', rw=True) as session:
    
    # 3. GENERATE an AES Key inside the HSM
    # The key never leaves the "hardware" boundary
    key = session.generate_key(KeyType.AES, 256, label="MyDemoKey")
    print(f"Key created successfully! ID: {key.label}")

    # 4. ENCRYPT data using the key inside the HSM
    data = b"Top Secret Data"
    iv = session.generate_random(128)  # AES block size
    crypttext = key.encrypt(data, mechanism_param=iv)
    
    print(f"Encrypted hex: {crypttext.hex()}")

    # 5. DECRYPT data
    decrypted = key.decrypt(crypttext, mechanism_param=iv)
    print(f"Decrypted text: {decrypted.decode()}")

Available tokens: ['DemoToken', '']
Connected to: DemoToken
Connecting to token: DemoToken
Key created successfully! ID: MyDemoKey
Encrypted hex: 857946cc404d45b117759de0dd75dec1
Decrypted text: Top Secret Data
